In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import argparse
import pickle

In [2]:
print("[INFO] loading face embeddings...")
data = pickle.loads(open('embeddings.pickle', "rb").read())

[INFO] loading face embeddings...


In [3]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.
C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\t

In [16]:
left_input=Input(shape=(1,128))
right_input=Input(shape=(1,128))
model=Sequential()
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32, activation='sigmoid'))
encoded_L=model(left_input)
encoded_R=model(right_input)
L1_layer=Lambda(lambda tensors:K.abs(tensors[0]-tensors[1]))
L1_distance=L1_layer([encoded_L,encoded_R])
prediction= Dense(1,activation='sigmoid')(L1_distance)
siamese_net=Model(inputs=[left_input,right_input],output=prediction)
siamese_net.compile(loss='binary_crossentropy',metrics=['acc'],optimizer='Adam')
siamese_net.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1, 128)       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1, 128)       0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 1, 32)        26848       input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 1, 32)        0           sequential_3[1][0]         

C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  if sys.path[0] == '':


i=0
j=0
data['embeddings'][i]
data['embeddings'][j]
data['names'][i]
data['names'][j]
if equal then y_train[count]=1
else y_train[count]=0
lets say we choose 50 random pairs
left_input,right_input append data['embeddings'][i] and data['embeddings'][j] respectively:
    
        
        

In [17]:
from random import randint
import numpy as np
def random_pair(data):
    y_train=np.zeros((50,1))
    left_input=[]
    right_input=[]
    for count in range(50):
        left=randint(0,49)
        right=randint(0,49)
        if(right!=left):
            if(data['names'][left]==data['names'][right]):
                y_train[count]=1
            else:
                y_train[count]=0
            if count==0:
                left_input=data['embeddings'][left]
                right_input=data['embeddings'][right]
            else:
                left_input=np.vstack((left_input,data['embeddings'][left]))
                right_input=np.vstack((right_input,data['embeddings'][right]))
        else:
            count=count-1
            continue
    return left_input,right_input,y_train

In [ ]:
left_input,right_input,y_train=random_pair(data)

In [19]:
left_input_reshape=left_input.reshape(50,1,128)
right_input_reshape=right_input.reshape(50,1,128)

(50, 1, 128)


In [20]:
y_train_reshape=y_train.reshape(50,1,1)

In [24]:
EPOCH=100
BATCH=20
siamese_net.fit([left_input_reshape,right_input_reshape],y_train_reshape,epochs=EPOCH,batch_size=BATCH)

Epoch 1/100
50/50 [==============================] - 0s 279us/step - loss: 0.6735 - acc: 0.7800
Epoch 2/100
50/50 [==============================] - 0s 239us/step - loss: 0.6515 - acc: 0.8000
Epoch 3/100
50/50 [==============================] - 0s 239us/step - loss: 0.6300 - acc: 0.8000
Epoch 4/100
50/50 [==============================] - 0s 239us/step - loss: 0.6032 - acc: 0.8000
Epoch 5/100
50/50 [==============================] - 0s 199us/step - loss: 0.5779 - acc: 0.8000
Epoch 6/100
50/50 [==============================] - 0s 239us/step - loss: 0.5471 - acc: 0.8000
Epoch 7/100
50/50 [==============================] - 0s 239us/step - loss: 0.5173 - acc: 0.8000
Epoch 8/100
50/50 [==============================] - 0s 199us/step - loss: 0.4866 - acc: 0.8000
Epoch 9/100
50/50 [==============================] - 0s 219us/step - loss: 0.4597 - acc: 0.8000
Epoch 10/100
50/50 [==============================] - 0s 220us/step - loss: 0.4321 - acc: 0.8000
Epoch 11/100
50/50 [===================

50/50 [==============================] - 0s 224us/step - loss: 0.0972 - acc: 1.0000
Epoch 86/100
50/50 [==============================] - 0s 200us/step - loss: 0.0957 - acc: 1.0000
Epoch 87/100
50/50 [==============================] - 0s 139us/step - loss: 0.0940 - acc: 1.0000
Epoch 88/100
50/50 [==============================] - 0s 180us/step - loss: 0.0933 - acc: 1.0000
Epoch 89/100
50/50 [==============================] - 0s 179us/step - loss: 0.0895 - acc: 1.0000
Epoch 90/100
50/50 [==============================] - 0s 139us/step - loss: 0.0907 - acc: 1.0000
Epoch 91/100
50/50 [==============================] - 0s 179us/step - loss: 0.0894 - acc: 1.0000
Epoch 92/100
50/50 [==============================] - 0s 199us/step - loss: 0.0845 - acc: 1.0000
Epoch 93/100
50/50 [==============================] - 0s 199us/step - loss: 0.0850 - acc: 1.0000
Epoch 94/100
50/50 [==============================] - 0s 160us/step - loss: 0.0823 - acc: 1.0000
Epoch 95/100
50/50 [=======================

In [25]:
# write the actual face recognition model to disk
f = open("recognizer.pickle", "wb")
f.write(pickle.dumps(siamese_net))
f.close()